In [1]:
import torch
from transformers import AutoImageProcessor, AutoModel
from PIL import Image
import requests
import os
#os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'

url = 'http://images.cocodataset.org/val2017/000000039769.jpg'
image = Image.open(requests.get(url, stream=True).raw)

processor = AutoImageProcessor.from_pretrained('facebook/dinov2-base')
model = AutoModel.from_pretrained('facebook/dinov2-base')

inputs = processor(images=image, return_tensors="pt")
outputs = model(**inputs)
last_hidden_states = outputs[0]

# We have to force return_dict=False for tracing
model.config.return_dict = False

with torch.no_grad():
    traced_model = torch.jit.trace(model, [inputs.pixel_values])
    traced_outputs = traced_model(inputs.pixel_values)

print((last_hidden_states - traced_outputs[0]).abs().max())

/home/yuhaowang/anaconda3/envs/gigapath/lib/python3.9/site-packages/torchvision/datapoints/__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 to learn more about the APIs that we suspect might involve future changes. You can silence this warning by calling torchvision.disable_beta_transforms_warning().
  warnings.warn(_BETA_TRANSFORMS_WARNING)
/home/yuhaowang/anaconda3/envs/gigapath/lib/python3.9/site-packages/torchvision/transforms/v2/__init__.py:54: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please

tensor(6.1512e-05, grad_fn=<MaxBackward1>)


In [2]:
import os
tile_dir='/home/yuhaowang/data/processed_data/'
os.listdir(tile_dir)

['TCGA-BRCA', 'TCGA-LUAD', 'TCGA-LUSC', 'TCGA-BLCA']

In [10]:
import os
os.listdir('/home/yuhaowang/data/processed_data/TCGA-LUAD/output')
for folder in os.listdir('/home/yuhaowang/data/processed_data/TCGA-LUAD/output'):
    #TCGA-44-7662-01Z-00-DX1.2f0b6cea-795a-40ad-93a9-319858e6fb3b.svs -> delete .svs
    if folder.endswith('.svs'):
        os.rename(os.path.join('/home/yuhaowang/data/processed_data/TCGA-LUAD/output', folder), os.path.join('/home/yuhaowang/data/processed_data/TCGA-LUAD/output', folder[:-4]))

In [ ]:
# from datasets import load_dataset

# dataset = load_dataset("imagefolder", data_dir="/home/yuhaowang/data/processed_data/TCGA-LUAD")
import torchvision
from torchvision.datasets import ImageFolder
from datasets import Dataset
dataset= ImageFolder("/home/yuhaowang/data/processed_data/TCGA-BRCA/output")

class TileDataset(ImageFolder):
    def __init__(self, root, transform=None, loader=Image.open):
        super(TileDataset, self).__init__(root, transform=transform,loader=loader)
        self.samples = [(os.path.join(root, x[0]), x[1]) for x in self.samples]

    def __getitem__(self, index):
        path, _ = self.samples[index]
        sample = self.loader(path)
        if self.transform is not None:  
            sample = self.transform(sample)
        result = {"pixel_values": sample}
        return result

from torchvision import transforms
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

dataset = TileDataset("/home/yuhaowang/data/processed_data/TCGA-LUAD/output", transform=transform)


NameError: name 'Image' is not defined

: 

In [3]:
from transformers import Trainer, TrainingArguments
import torch
from transformers import AutoImageProcessor, AutoModel
from PIL import Image
import requests
import os
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
)

model = AutoModel.from_pretrained('facebook/dinov2-base')

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=dataset,         # training dataset
    eval_dataset=dataset             # evaluation dataset
)

trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:w

Abort: 